### JSTR for Streamlit



In [1]:
import pandas as pd
import numpy as np
import helper as hp

#### Part 1.2 Read in CSV uploaded by user in Streamlit, whereby:
#### 1st colum: job role
#### 2nd column: job description
#### 3rd column: skills

In [2]:
## Codes to read in input file via streamlit and save as "uploaded_dataset.csv"
## Minimally, 1st col = job title/archetype
## 2nd col = job description
## 3rd col (optional) = skills. if 3rd col is missing, then skills are inferred from 2nd col.

In [3]:
#################################################################################
# For Jupyter Notebook, Assume "uploaded_dataset.csv" is uploaded from Streamlit#
#################################################################################
df = pd.read_csv("data_raw/uploaded_dataset_short.csv")
df_uploaded = df.copy()

In [4]:
# Read in all SFw job roles with their associated CWF Clusters 
df_cwf = pd.read_excel('data_raw/job role_cwf.xlsx')
# Load cwf kt reference table (this is an output from preprocessing step 0.1)
cwf_kt = pd.read_csv("data_processed/cwf_kt.csv")
# Read SFw Database - Job Role (Key) (To extract Average TSC Levels - use it to filter unrealistic adjacent pairs)
sfw_df = pd.read_excel('data_raw\SFw Database_27Jul_2022.xlsx', sheet_name='Job Role (Key)')
# Load SSOC->SFw Job Role Reference Table
df_ssoc = pd.read_excel('data_raw\SFw Database_27Jul_2022.xlsx', sheet_name='Job Role_SSOC')
# Load SSOC->Wages Reference Table
df_salary = pd.read_csv('data_raw\wages_2022.csv')
# Load SSOC->Demand Reference Table
df_demand = pd.read_csv('data_raw\jobdemand_2022.csv')
# Load 11K->2K Skills SEA Mapping Table
df_sea = pd.read_excel("data_raw/2k_to_11k_mapping_include_skills_tag_20220914_SIPD_V02.xlsx", sheet_name="2k_to_11k_mapping_include_skill")
# Load SFw Job Role->TSC Mapping Table
df_sfw = pd.read_excel("data_raw/SFw Database_27Jul_2022.xlsx", sheet_name="Job Role_TSC")
# Load item-job_roles IDs
df_item_jobroles = pd.read_csv("data_raw/item_job_roles.csv")
# Load skills_demand table
df_skill_demand = pd.read_csv("data_raw/stats_year_ssoc_tsc.csv")

In [5]:
# Extract values from the first column in the uploaded data and populate the uploaded_job_titles list
uploaded_job_titles = []
uploaded_job_titles = df.iloc[:, 0].tolist()

#### Part 1.3 Add raw job description for the uploaded job roles in 'cwf_kt.csv', find most similar cwf clusters for each job roles and stores output to dataframe list

In [6]:
# Add raw job description for the uploaded job roles in 'cwf_kt.csv', 
# find most similar cwf clusters for each job roles and stores output to dataframe list
# Create an empty list to store cwf_kt DataFrames for all the job titles in uploaded dataset
all_cwf_kt_df = []

# Loop through each row and call generate_similar_cwf_kt() for the value in the first column
for index, row in df.iterrows():
    # Check if job_title is NaN
    # if pd.isna(row.iloc[0]):
    #    break
    job_title = row.iloc[0]
    job_desc = row.iloc[1]
    result_df = hp.generate_similar_cwf_kt(index, job_title, job_desc)
    all_cwf_kt_df.append(result_df)

#### Part 1.4 - Populate all_cwf_kt_df [ ] with Function ID, Sector, Track, Job role, L1/2/3 clusters

In [7]:
#def populateCWFClusters(df):

In [8]:
# Populate all_cwf_kt_df [ ] with Function ID, Sector, Track, Job role, L1/2/3 clusters
# Create an empty list to store the resulting DataFrames
populated_df = []

# Iterate over the list of DataFrames and apply the function
for df in all_cwf_kt_df:
    result_df = hp.populateCWFClusters(df, cwf_kt, df_cwf)
    populated_df.append(result_df) 

#### Part 2 Prepare df_jobrole_cwf for JSTR

In [9]:
df_jobrole_cwf = hp.prepareJobroleCWF(df_cwf, populated_df, uploaded_job_titles)

#### Part 3.1 - Data preparation to Run JSTR

In [10]:
#dataPrepForJSTR(df_jobrole_cwf)

In [11]:
JobCWF_df = hp.dataPrepForJSTR(df_jobrole_cwf)

#### Part 3.2 - Compute Job Adjacency Scores based on CWF Clusters

In [12]:
#computeJobAdjacency(df_job_cwf)
df_cwf = hp.computeJobAdjacency(JobCWF_df)

#### Part 3.3 - Populate job pairs with job title

In [13]:
# populateJobPairsWithJobTitle(df_jobrole_cwf, df_cwf, sfw_df)

In [14]:
df_cwf = hp.populateJobPairsWithJobTitle(df_jobrole_cwf, df_cwf, sfw_df)

### 3.4 Populate SSOC for destination jobs

In [15]:
# populateSSOCforDestJobs(df_cwf, df_ssoc)

In [16]:
df_cwf = hp.populateSSOCforDestJobs(df_cwf, df_ssoc)

#### 3.5 Populate Jobtech salary data

In [17]:
# populateSalaryforDestJobs(df_cwf, df_salary)

In [18]:
df_cwf = hp.populateSalaryforDestJobs(df_cwf, df_salary)

#### 3.6 Populate Jobtech job demand data

In [19]:
# hp.populateDemandforDestJobs(df_cwf, df_demand)

In [20]:
df_cwf = hp.populateDemandforDestJobs(df_cwf, df_demand)

#### 4.1 Extract Skills from Uploaded Dataset

In [21]:
# extractSkillsfromUploadedFile (df_uploaded, df_skills)

In [22]:
df_skills = pd.DataFrame()
df_skills = hp.extractSkillsfromUploadedFile(df_uploaded, df_skills)

#### 4.2 Convert SFw Database JobRole-to-TSC to JobRole-to-SEA

In [23]:
#convertTSCtoSEA (df_sea, df_sfw, df_item_jobroles)

In [24]:
df_sfw = hp.convertTSCtoSEA (df_sea, df_sfw, df_item_jobroles)

#### 4.3 Load JSTR Output and populate the skills data

#### 4.4 Merge JSTR Output with SEA Skills

In [25]:
# mapDestSSOC(df_cwf, df_ssoc)
df_cwf = hp.mergeOutputwithSEASkills(df_cwf, df_ssoc, df_sfw, df_skills, df_skill_demand)

In [26]:
df_cwf.to_csv("data_processed/jstr_output.csv", index=False)